---
## 01_Dataload_Clean
---

# Notebook 01: Data Load & Clean

This notebook loads raw World Bank and UN datasets, reshapes them into long format, and performs initial cleaning steps.  
The cleaned datasets are exported into `/data/clean/` for use in later notebooks.

### Steps
1. Load raw CSVs  
2. Reshape wide → long  
3. Standardize columns  
4. Filter years ≥ 1990  
5. Export cleaned versions to `/data/clean/`  

## Imports
Load required Python libraries for data cleaning and reshaping.

In [ ]:
import pandas as pd
import numpy as np
import os

## Define Data Directories and Files
Set up input (`/data/raw/`) and output (`/data/clean/`) directories, and specify filenames for each dataset.

In [ ]:
raw_dir = "../data/raw/"
clean_dir = "../data/clean/"

os.makedirs(clean_dir, exist_ok=True)

files = {
    "electricity": "API_EG.ELC.ACCS.ZS_DS2_en_csv_v2_38353.csv",
    "internet": "API_IT.NET.USER.ZS_DS2_en_csv_v2_112825.csv",
    "mobile": "API_IT.CEL.SETS.P2_DS2_en_csv_v2_37045.csv",
    "literacy": "API_SE.ADT.LITR.ZS_DS2_en_csv_v2_37553.csv",
    "tertiary": "API_SE.TER.ENRR_DS2_en_csv_v2_399842.csv",
    "researchers": "API_SP.POP.SCIE.RD.P6_DS2_en_csv_v2_463385.csv",
    "rnd_gdp": "API_GB.XPD.RSDV.GD.ZS_DS2_en_csv_v2_464376.csv",
    "gdp_ppp": "API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2_406489.csv",
    "gov_effect": "5a88699c-e86d-490a-8c9e-c070313913df_Data.csv"
}

## Load Raw Data into DataFrames
Read each dataset into a dictionary of DataFrames.  
- Most World Bank CSVs require `skiprows=4`.  
- The government effectiveness dataset is loaded directly.  
Each dataset’s shape is printed for confirmation.

In [ ]:
raw_dfs = {}
for key, fname in files.items():
    path = os.path.join(raw_dir, fname)
    try:
        if key == "gov_effect":
            df = pd.read_csv(path)  # no skiprows for gov_effect
        else:
            df = pd.read_csv(path, skiprows=4)  # World Bank usually needs skiprows=4
        raw_dfs[key] = df
        print(f"Loaded {key}: {df.shape}")
    except Exception as e:
        print(f"Failed to load {key}: {e}")

✔ Loaded electricity: (266, 70)
✔ Loaded internet: (266, 70)
✔ Loaded mobile: (266, 70)
✔ Loaded literacy: (266, 70)
✔ Loaded tertiary: (266, 70)
✔ Loaded researchers: (266, 70)
✔ Loaded rnd_gdp: (266, 70)
✔ Loaded gdp_ppp: (266, 70)
✔ Loaded gov_effect: (7709, 29)


## Define Cleaning Function
Helper function to standardize World Bank-style indicator files:
- Reshape from wide to long format  
- Rename and standardize columns  
- Filter to years ≥ 1990  
- Retain NaN values for later handling

In [ ]:
def clean_wb_df(df, indicator_name):
    """
    Cleans World Bank-style indicator files:
    - Melt wide → long
    - Rename columns
    - Filter year >= 1990
    - Retain NaNs
    """
    df_long = df.melt(
        id_vars=["Country Name", "Country Code"],
        var_name="Year", value_name="Value"
    )
    df_long["Year"] = pd.to_numeric(df_long["Year"], errors="coerce")
    df_long = df_long[df_long["Year"] >= 1990]
    df_long["Indicator"] = indicator_name
    return df_long[["Country Name", "Country Code", "Year", "Indicator", "Value"]]

## Apply Cleaning Function
Apply the helper function to all World Bank–style datasets (electricity, internet, mobile, literacy, tertiary, researchers, R&D, GDP PPP).  
The government effectiveness dataset is excluded here and will be handled separately.

In [ ]:
clean_dfs = {}

for key, df in raw_dfs.items():
    if key == "gov_effect":
        # Handle separately later
        continue
    cleaned = clean_wb_df(df, key)
    clean_dfs[key] = cleaned
    print(f"Cleaned {key}: {cleaned.shape}")

✔ Cleaned electricity: (9310, 5)
✔ Cleaned internet: (9310, 5)
✔ Cleaned mobile: (9310, 5)
✔ Cleaned literacy: (9310, 5)
✔ Cleaned tertiary: (9310, 5)
✔ Cleaned researchers: (9310, 5)
✔ Cleaned rnd_gdp: (9310, 5)
✔ Cleaned gdp_ppp: (9310, 5)


## Clean Government Effectiveness (Special Case)
The government effectiveness dataset is structured differently than World Bank-style files.  
Steps:
- Filter to the *"Government Effectiveness: Estimate"* series  
- Reshape from wide to long format  
- Extract numeric years (e.g., `1996 [YR1996]` → `1996`)  
- Filter for years ≥ 1990  
- Tag with the `gov_effect` indicator name  

In [ ]:
gov_df = raw_dfs["gov_effect"].copy()

gov_df = gov_df[gov_df["Series Name"] == "Government Effectiveness: Estimate"]

gov_long = gov_df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year", value_name="Value"
)

gov_long["Year"] = gov_long["Year"].str.extract(r"(\d{4})").astype(float)

gov_long = gov_long[gov_long["Year"] >= 1990]

gov_long["Indicator"] = "gov_effect"

gov_long = gov_long[["Country Name", "Country Code", "Year", "Indicator", "Value"]]

clean_dfs["gov_effect"] = gov_long
print(f"Cleaned gov_effect (filtered): {gov_long.shape}")

✔ Cleaned gov_effect (filtered): (5350, 5)


## Save Cleaned Data
Export each cleaned dataset into the `/data/clean/` directory.  
Each file is saved with the format `{indicator}_clean.csv`.

In [ ]:
for key, df in clean_dfs.items():
    out_path = os.path.join(clean_dir, f"{key}_clean.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved: {out_path}")

Saved: ../data/clean/electricity_clean.csv
Saved: ../data/clean/internet_clean.csv
Saved: ../data/clean/mobile_clean.csv
Saved: ../data/clean/literacy_clean.csv
Saved: ../data/clean/tertiary_clean.csv
Saved: ../data/clean/researchers_clean.csv
Saved: ../data/clean/rnd_gdp_clean.csv
Saved: ../data/clean/gdp_ppp_clean.csv
Saved: ../data/clean/gov_effect_clean.csv


---
## Wrap-Up
---
- Successfully loaded raw World Bank and UN datasets.  
- Applied cleaning function to World Bank-style indicators and a custom cleaning routine for Government Effectiveness
- Exported cleaned datasets into `/data/clean/` with standardized formats.  
- These outputs are now ready for normalization and scaling in 
**Notebook 03: Normalize & Scale**.  